# Response Model - Return Type

In [2]:
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()

In [3]:
class Item(BaseModel):
    name: str
    description: str | None = None
    price: float
    tax: float | None = None
    tags: list[str] = []

you can declare `response type` by annotating the path operation function `return type`

In [6]:
@app.post("/items/")
async def create_item(item: Item) -> Item: # '-> Item' is annotating the return type 
    return item

you can also use `type annotations` for the `return type` 

In [7]:
@app.get("/items/")
async def read_items() -> list[Item]: 
    return [
        Item(name="Portal Gun", price=42.0),
        Item(name="Plumbus", price=32.0),
    ]

when declaring the response type, the fastapi will:

- validate the returned data
- add a json schema for the response (for api doc)
- it will limit and filter the output data the output data to what is defined in the returned type (important for security)

## `response_model` Parameter

You can use the response_model parameter in FastAPI's path operation decorator instead of return type annotations when you want to return data (like a dictionary or database object) that differs from the declared Pydantic model type, allowing the model to handle documentation and validation while avoiding type checker errors.


• Use `response_model` parameter in FastAPI's path operation decorator instead of return type annotations

• Useful when returning data that differs from the declared type (e.g., returning a dictionary but declaring a Pydantic model)

• Allows the Pydantic model to handle documentation and validation for the returned object

• Avoids type checker errors that would occur when the actual return type doesn't match the declared type annotation

• Common scenarios include returning dictionaries, database objects, or other types while maintaining Pydantic model benefits


You can use the response_model parameter in any of the path operations:

- @app.get()
- @app.post()
- @app.put()
- @app.delete()
- etc.


In [9]:
from typing import Any

In [10]:
@app.post("/items/", response_model=Item)
async def create_item(item: Item) -> Any:
    return item


@app.get("/items/", response_model=list[Item])
async def read_items() -> Any:
    return [
        {"name": "Portal Gun", "price": 42.0},
        {"name": "Plumbus", "price": 32.0},
    ]

FastAPI will use this `response_model` to do all the data documentation, validation, etc. and also to convert and filter the output data to its type declaration.

we can create two different models for getting the input and outputting

In [12]:
from pydantic import EmailStr

In [14]:
class UserIn(BaseModel): 
    username: str 
    password: str 
    email: EmailStr 
    full_name: str | None = None

In [15]:
class UserOut(BaseModel):
    username: str 
    email: EmailStr 
    full_name: str | None = None

In [16]:
@app.post("/user/", response_model=UserOut) 
async def create_user(user: UserIn) -> Any:
    return user

this way, we dont return the password

# Return Type and Data Filtering

- in the above example, we had to declare two models and use `response_model` parameter to not return `password` data.
- doing it like above, means that we dont get support from the editor and tools checking the function return type.
- Since we want the model to `filter/remove` some data, we can use `classes and inheritance` to take advantage of function type annotations to get better support in the editor and tools, and still get the FastAPI data filtering.

In [17]:
class BaseUser(BaseModel): 
    username: str 
    email: EmailStr 
    full_name: str | None = None 

class UserIn(BaseUser): 
    password: str 

@app.post("/user/") 
async def create_user(user: UserIn) -> BaseUser: 
    return user 

This way, you can get the best of both worlds: `type annotations` with `tooling support` and `data filtering`.

## Other Return Type Annotations

- Return a Response Directly
- Annotate a Response Subclass
- Invalid Return Type Annotations
- Disable Response Model


### Return a Response Directly

In [18]:
from fastapi import FastAPI, Response
from fastapi.responses import JSONResponse, RedirectResponse

app = FastAPI()


@app.get("/portal")
async def get_portal(teleport: bool = False) -> Response:
    if teleport:
        return RedirectResponse(url="https://www.youtube.com/watch?v=dQw4w9WgXcQ")
    return JSONResponse(content={"message": "Here's your interdimensional portal."})

### Annotate a Response Subclass

In [22]:
from fastapi import FastAPI
from fastapi.responses import RedirectResponse

app = FastAPI()


@app.get("/teleport")
async def get_teleport() -> RedirectResponse:
    return RedirectResponse(url="https://www.youtube.com/watch?v=dQw4w9WgXcQ")

### Invalid Return Type Annotations 

- when you return some other arbitrary object that is not a valid Pydantic type (e.g. a database object) and you annotate it like that in the function, FastAPI will try to create a Pydantic response model from that type annotation, and will fail.

In [23]:
from fastapi import FastAPI, Response
from fastapi.responses import RedirectResponse

app = FastAPI()


@app.get("/portal")
async def get_portal(teleport: bool = False) -> Response | dict: # this will fail
    if teleport:
        return RedirectResponse(url="https://www.youtube.com/watch?v=dQw4w9WgXcQ")
    return {"message": "Here's your interdimensional portal."}

FastAPIError: Invalid args for response field! Hint: check that starlette.responses.Response | dict is a valid Pydantic field type. If you are using a return type annotation that is not a valid Pydantic field (e.g. Union[Response, dict, None]) you can disable generating the response model from the type annotation with the path operation decorator parameter response_model=None. Read more: https://fastapi.tiangolo.com/tutorial/response-model/

### Disable Response Model

you might not want to have the default data validation, documentation, filtering, etc. that is performed by FastAPI.

But you might want to still keep the return type annotation in the function to get the support from tools like editors and type checkers (e.g. mypy)

In [25]:
from fastapi import FastAPI, Response
from fastapi.responses import RedirectResponse

app = FastAPI()


@app.get("/portal", response_model=None)
async def get_portal(teleport: bool = False) -> Response | dict:
    if teleport:
        return RedirectResponse(url="https://www.youtube.com/watch?v=dQw4w9WgXcQ")
    return {"message": "Here's your interdimensional portal."}

## Response Model encoding parameters


- Use the response_model_exclude_unset parameter
- Data with values for fields with defaults
- Data with the same values as the defaults

- Using lists instead of sets

## Response Model encoding parameters 

Your response model could have default values

In [26]:
class Item(BaseModel):
    name: str
    description: str | None = None
    price: float
    tax: float = 10.5
    tags: list[str] = []

In [28]:

items = {
    "foo": {"name": "Foo", "price": 50.2},
    "bar": {"name": "Bar", "description": "The bartenders", "price": 62, "tax": 20.2},
    "baz": {"name": "Baz", "description": None, "price": 50.2, "tax": 10.5, "tags": []},
}


@app.get("/items/{item_id}", response_model=Item, response_model_exclude_unset=True)
async def read_item(item_id: str):
    """
    {
        "name": "Foo",
        "price": 50.2
    }
    """
    return items[item_id]


setting `response_model_exclude_unset=True` will make those default values won't be included in the response, only the values actually set.

You can also use the path operation decorator parameters `response_model_include` and `response_model_exclude`.